# 🍓 Strawberry Vision – Google Colab Başlangıç

Bu notebook, Strawberry Vision projesini Google Colab'de çalıştırmak için kapsamlı bir kılavuzdur.

## 📋 İçerik
1. **Kurulum**: Bağımlılıklar ve proje yapısı
2. **Dataset Hazırlama**: Roboflow'dan indirme ve etiketleme
3. **Model Eğitimi**: YOLOv8 eğitimi
4. **Inference**: Eğitilmiş model ile tahmin
5. **Değerlendirme**: Model performans analizi

## 🚀 Hızlı Başlangıç
Sadece inference yapmak istiyorsanız bölüm 1 ve 4'ü çalıştırın.
Tam eğitim pipeline için tüm bölümleri sırayla çalıştırın.

In [ ]:
# 1. KURULUM - Bağımlılıkları Yükle
print("📦 Bağımlılıklar yükleniyor...")

!pip install -q ultralytics>=8.0.0
!pip install -q opencv-python>=4.8.0
!pip install -q numpy>=1.24.0
!pip install -q matplotlib>=3.7.0
!pip install -q roboflow>=1.1.0
!pip install -q albumentations>=1.3.0
!pip install -q pyyaml>=6.0

print("✅ Bağımlılıklar yüklendi!")

In [ ]:
# 2. PROJE YAPISINI KONTROL ET
import os
from pathlib import Path

# Proje root dizini
project_root = Path.cwd()
print(f"📁 Proje dizini: {project_root}")

# Gerekli klasörleri kontrol et
required_dirs = [
    "strawberry_vision",
    "configs",
    "scripts",
    "tests",
    "docs"
]

for dir_name in required_dirs:
    dir_path = project_root / dir_name
    if dir_path.exists():
        print(f"✅ {dir_name}/ bulundu")
    else:
        print(f"⚠️  {dir_name}/ bulunamadı - oluşturuluyor...")
        dir_path.mkdir(exist_ok=True)

# Dataset klasörü oluştur
(project_root / "datasets").mkdir(exist_ok=True)
print("✅ Proje yapısı hazır!")

## 📦 Bölüm 2: Dataset Hazırlama

### Roboflow'dan Dataset İndirme
Roboflow API key'inizi alın: https://app.roboflow.com/settings/api

In [ ]:
# 2.1. Roboflow'dan Dataset İndir (Opsiyonel - kendi datasetiniz varsa atlayın)
# API key'inizi buraya girin
ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"  # Değiştirin!
WORKSPACE = "strawberry"
PROJECT = "ripeness"
VERSION = 1

if ROBOFLOW_API_KEY != "YOUR_API_KEY_HERE":
    print("📥 Roboflow'dan dataset indiriliyor...")
    !python scripts/download_dataset.py \
        --api-key {ROBOFLOW_API_KEY} \
        --workspace {WORKSPACE} \
        --project {PROJECT} \
        --version {VERSION} \
        --output datasets/roboflow
    print("✅ Dataset indirildi!")
else:
    print("⚠️  API key girilmedi, bu adım atlandı.")

In [ ]:
# 2.2. Sınıf Etiketlerini Güncelle
# Roboflow'dan gelen sınıf isimlerini standart formata çevir
print("🔄 Sınıf etiketleri güncelleniyor...")

!python scripts/relabel_dataset.py \
    --input datasets/roboflow \
    --output datasets/processed

print("✅ Sınıf etiketleri güncellendi!")
print("Standart sınıflar: strawberry_ripe, strawberry_semi_ripe, strawberry_unripe")

In [ ]:
# 2.3. Augmentation Uygula (Opsiyonel)
# Dataset boyutunu artırmak için augmentation uygulayın
APPLY_AUGMENTATION = False  # True yapın augmentation için
AUGMENTATION_FACTOR = 2  # Her görüntü için kaç augmented versiyon

if APPLY_AUGMENTATION:
    print("🎨 Augmentation uygulanıyor...")
    !python scripts/augment_dataset.py \
        --input datasets/processed \
        --output datasets/augmented \
        --factor {AUGMENTATION_FACTOR}
    
    DATASET_PATH = "datasets/augmented"
    print(f"✅ Augmentation tamamlandı! Dataset: {DATASET_PATH}")
else:
    DATASET_PATH = "datasets/processed"
    print(f"ℹ️  Augmentation atlandı. Dataset: {DATASET_PATH}")

## 🎓 Bölüm 3: Model Eğitimi

### YOLOv8 ile Çilek Tespiti ve Olgunluk Sınıflandırması

In [ ]:
# 3.1. Eğitim Parametrelerini Ayarla
# Config dosyası veya doğrudan parametreler kullanabilirsiniz

# Eğitim parametreleri
MODEL = "yolov8n.pt"  # yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
EPOCHS = 50  # Colab için 50-100 epoch önerilir
BATCH_SIZE = 16  # GPU belleğine göre ayarlayın (8, 16, 32)
IMAGE_SIZE = 640
EXPERIMENT_NAME = "strawberry_colab_exp"

print(f"🎯 Eğitim Parametreleri:")
print(f"  Model: {MODEL}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Image Size: {IMAGE_SIZE}")

In [ ]:
# 3.2. Model Eğitimi Başlat
print("🚀 Model eğitimi başlıyor...")
print("⏱️  Bu işlem GPU ile 30-60 dakika sürebilir.")

!python scripts/train_yolo.py \
    --data {DATASET_PATH}/data.yaml \
    --model {MODEL} \
    --epochs {EPOCHS} \
    --batch {BATCH_SIZE} \
    --imgsz {IMAGE_SIZE} \
    --name {EXPERIMENT_NAME} \
    --device 0

print("✅ Eğitim tamamlandı!")
print(f"📊 Sonuçlar: runs/train/{EXPERIMENT_NAME}")
print(f"🏆 En iyi model: runs/train/{EXPERIMENT_NAME}/weights/best.pt")

In [ ]:
# 3.3. Eğitim Sonuçlarını Görselleştir
from IPython.display import Image, display
import matplotlib.pyplot as plt

# Eğitim grafikleri
results_path = f"runs/train/{EXPERIMENT_NAME}"

print("📊 Eğitim Grafikleri:")
try:
    display(Image(filename=f"{results_path}/results.png"))
    display(Image(filename=f"{results_path}/confusion_matrix.png"))
    display(Image(filename=f"{results_path}/val_batch0_pred.jpg"))
except:
    print("⚠️  Grafikler henüz oluşturulmadı veya bulunamadı.")

## 🔍 Bölüm 4: Inference (Tahmin)

### Eğitilmiş Model ile Çilek Tespiti

In [ ]:
# 4.1. Model Yolu Belirle
# Eğitim yaptıysanız:
TRAINED_MODEL = f"runs/train/{EXPERIMENT_NAME}/weights/best.pt"

# Veya önceden eğitilmiş model kullanın:
# TRAINED_MODEL = "path/to/your/best.pt"

# Model varlığını kontrol et
import os
if os.path.exists(TRAINED_MODEL):
    print(f"✅ Model bulundu: {TRAINED_MODEL}")
else:
    print(f"⚠️  Model bulunamadı: {TRAINED_MODEL}")
    print("Lütfen önce model eğitimi yapın veya model yolunu güncelleyin.")

In [ ]:
# 4.2. Tek Görüntü ile Inference
from strawberry_vision.application.pipeline import InferencePipeline
from strawberry_vision.infrastructure.detectors import YOLODetector
from strawberry_vision.infrastructure.sources import ImageSource
import cv2
import matplotlib.pyplot as plt

# Test görüntüsü yolu (kendi görüntünüzü yükleyin)
TEST_IMAGE = "sample.jpg"  # Değiştirin!

if os.path.exists(TEST_IMAGE) and os.path.exists(TRAINED_MODEL):
    print(f"🔍 Inference çalıştırılıyor: {TEST_IMAGE}")
    
    # Pipeline oluştur
    detector = YOLODetector(model_path=TRAINED_MODEL)
    pipeline = InferencePipeline(detector=detector, enable_logging=False)
    
    # Görüntüyü işle
    source = ImageSource(image_path=TEST_IMAGE)
    
    for frame in source:
        result = pipeline.run(frame)
        
        # Sonuçları göster
        print("\n📊 Tespit Sonuçları:")
        print(f"  Toplam çilek: {result.total}")
        print(f"  Olgunluk dağılımı: {result.counts}")
        print(f"  İşlem süresi: {result.processing_time:.3f}s")
        
        # Görselleştir
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Tespit: {result.total} çilek | {result.counts}")
        plt.show()
        
    # Metrikleri göster
    metrics = pipeline.get_metrics()
    print(f"\n📈 Pipeline Metrikleri:")
    print(f"  FPS: {metrics['fps']}")
    print(f"  Toplam tracked: {metrics['total_tracked']}")
else:
    print("⚠️  Test görüntüsü veya model bulunamadı!")

In [ ]:
# 4.3. Video ile Inference (Opsiyonel)
from strawberry_vision.infrastructure.sources import VideoSource

VIDEO_PATH = "test_video.mp4"  # Video yolunuzu girin
MAX_FRAMES = 50  # İşlenecek maksimum frame sayısı

if os.path.exists(VIDEO_PATH) and os.path.exists(TRAINED_MODEL):
    print(f"🎬 Video inference başlıyor: {VIDEO_PATH}")
    
    detector = YOLODetector(model_path=TRAINED_MODEL)
    pipeline = InferencePipeline(detector=detector, enable_logging=False)
    
    source = VideoSource(video_path=VIDEO_PATH, max_frames=MAX_FRAMES)
    
    frame_count = 0
    for frame in source:
        result = pipeline.run(frame)
        frame_count += 1
        
        if frame_count % 10 == 0:
            print(f"Frame {frame_count}: {result.total} çilek tespit edildi")
    
    # Final metrikler
    metrics = pipeline.get_metrics()
    print(f"\n✅ Video işleme tamamlandı!")
    print(f"  Toplam frame: {metrics['total_frames']}")
    print(f"  Ortalama FPS: {metrics['fps']}")
    print(f"  Toplam tracked: {metrics['total_tracked']}")
else:
    print("ℹ️  Video inference atlandı (video veya model bulunamadı)")

## 📊 Bölüm 5: Model Değerlendirme

### Validation/Test Seti Üzerinde Performans Analizi